In [1]:
from policyengine.countries import US
from policyengine.utils.charts import plotly_json_to_fig
from blank_slate_ubi_us.common import blank_slate_ubi, prepare_simulation

us = US()

blank_slate = blank_slate_ubi()
blank_slate_baseline = prepare_simulation()
blank_slate_reform = dict(reform=blank_slate, baseline_reform=blank_slate_baseline)

baseline, reformed = us._get_microsimulations(blank_slate_reform)

charts = us.population_reform(blank_slate)

In [2]:
def format_fig(fig):
    fig.update_xaxes(
        title_font=dict(size=16, color="black"), tickfont={"size": 14}
    )
    fig.update_yaxes(
        title_font=dict(size=16, color="black"), tickfont={"size": 14}
    )
    fig.update_layout(
        hoverlabel_align="right",
        font_family="Roboto",
        title_font_size=20,
        plot_bgcolor="white",
        paper_bgcolor="white",
        width=800,
        height=600,
    )
    return fig


In [3]:
import pandas as pd

change = lambda variable: reformed.calc(variable).sum() - baseline.calc(variable).sum()

existing_tax_changes = sum([baseline.calc(variable).sum() for variable in [
    "spm_unit_federal_tax",
    "spm_unit_payroll_tax",
    "spm_unit_self_employment_tax",
]])

budget = pd.DataFrame({
    "Name": [
        "Abolish federal income tax",
        "Abolish payroll taxes",
        "Flat income tax",
        "Total tax reforms",
        "Repeal SNAP normal allotment",
        "Abolish SSI",
        "Abolish WIC",
        "Abolish TANF",
        "Abolish housing subsidies",
        "Universal basic income",
        "Total benefit reforms",
        "Total tax-benefit reforms",
    ],
    "Revenue effect": [
        -baseline.calc("spm_unit_federal_tax").sum(),
        -baseline.calc("spm_unit_payroll_tax").sum() + baseline.calc("spm_unit_self_employment_tax").sum(),
        change("spm_unit_taxes") + existing_tax_changes,
        change("spm_unit_taxes"),
        change("snap"),
        change("ssi"),
        change("wic"),
        change("tanf"),
        change("spm_unit_capped_housing_subsidy"),
        -reformed.calc("ubi").sum(),
        change("spm_unit_benefits"),
        change("spm_unit_net_income"),
    ],
})
budget["Revenue effect"] = budget["Revenue effect"].apply(lambda x: round(x / 1e9))
budget

,Name,Revenue effect
0,Abolish federal income tax,-1272
1,Abolish payroll taxes,-582
2,Flat income tax,5458
3,Total tax reforms,3494
4,Repeal SNAP normal allotment,-98
5,Abolish SSI,-63
6,Abolish WIC,-5
7,Abolish TANF,-6
8,Abolish housing subsidies,-24
9,Universal basic income,-3764


## Figure 1

In [4]:
format_fig(plotly_json_to_fig(charts["intra_income_decile_chart"]).update_layout(
    title="Distribution of gains and losses by income decile under Blank Slate UBI",
))

## Figure 2

In [5]:
from blank_slate_ubi_us.charts.age_winners import age_winner_chart

format_fig(plotly_json_to_fig(age_winner_chart(baseline, reformed, us.results_config)).update_layout(
    title="Distribution of gains and losses by age under Blank Slate UBI",
))

## Figure 3

In [6]:
from blank_slate_ubi_us.charts.state_choropleth import us_state_choropleth

us_state_choropleth(baseline, reformed)

## Figure 4

In [7]:
format_fig(plotly_json_to_fig(charts["poverty_chart"]).update_layout(
    title="Change to poverty rates by age group under Blank Slate UBI",
))

In [8]:
import numpy as np
poverty_rate = lambda sim: sim.calc("spm_unit_is_in_spm_poverty", map_to="person").mean()
deep_poverty_rate = lambda sim: sim.calc("spm_unit_is_in_deep_spm_poverty", map_to="person").mean()

def poverty_gap_fn(sim, deep):
    distance = sim.calc("spm_unit_spm_threshold") * (1 if not deep else 0.5) - sim.calc("spm_unit_net_income")
    positive_distance = np.maximum(0, distance)
    weight = sim.calc("spm_unit_weight").values
    return (weight * positive_distance).sum()

poverty_gap = lambda sim: poverty_gap_fn(sim, False)
deep_poverty_gap = lambda sim: poverty_gap_fn(sim, True)
metric_names = ["Poverty rate", "Deep poverty rate", "Poverty gap", "Deep poverty gap"]

import pandas as pd

# For each metric, add columns "Name", "Baseline", "Reformed", "Absolute change", "Relative change".

df = pd.DataFrame({
    "Name": metric_names,
    "Baseline": [poverty_rate(baseline) / 1e-2, deep_poverty_rate(baseline) / 1e-2, poverty_gap(baseline) / 1e9, deep_poverty_gap(baseline) / 1e9],
    "Reformed": [poverty_rate(reformed) / 1e-2, deep_poverty_rate(reformed) / 1e-2, poverty_gap(reformed) / 1e9, deep_poverty_gap(reformed) / 1e9],
})

df["Change"] = df.Reformed - df.Baseline
df["Relative change"] = (df.Reformed / df.Baseline - 1) * 100

df.Baseline = df.Baseline.round(1)
df.Reformed = df.Reformed.round(1)
df.Change = df.Change.round(1)
df["Relative change"] = df["Relative change"].round(1)

df

,Name,Baseline,Reformed,Change,Relative change
0,Poverty rate,11.3,2.4,-8.9,-79.1
1,Deep poverty rate,3.2,0.2,-3.0,-93.7
2,Poverty gap,130.9,21.6,-109.3,-83.5
3,Deep poverty gap,23.2,1.8,-21.3,-92.1


In [9]:
format_fig(plotly_json_to_fig(charts["rel_income_decile_chart"]).update_layout(
    title="Change to net income by income decile under Blank Slate UBI",
))

In [10]:
format_fig(plotly_json_to_fig(charts["inequality_chart"]).update_layout(
    title="Changes to inequality metrics under Blank Slate UBI",
))

In [11]:
from blank_slate_ubi_us.charts.state_poverty_choropleth import us_state_poverty_choropleth

us_state_poverty_choropleth(baseline, reformed)

In [12]:
from blank_slate_ubi_us.charts.program_winners import program_winner_chart

program_winner_chart(baseline, reformed, us.results_config)

In [13]:
household = {
    "people": {
        "adult": {
            "age": {2022: 25},
        },
        "adult_2": {
            "age": {2022: 25},
        },
        "child": {
            "age": {2022: 5},
        }
    },
    "tax_units": {
        "tax_unit": {
            "members": ["adult", "adult_2", "child"],
        }
    },
    "marital_units": {
        "marital_unit": {
            "members": ["adult", "adult_2"],
        }
    },
    "spm_units": {
        "spm_unit": {
            "members": ["adult", "adult_2", "child"],
        }
    },
    "households": {
        "household": {
            "members": ["adult", "adult_2", "child"],
            "state_code": {2022: "MA"},
            "zip_code": {2022: "01001"},
        }
    },
}

fig = us.household_variation(dict(reform=blank_slate, baseline_reform=blank_slate_baseline, household=household))

In [14]:
format_fig(plotly_json_to_fig(fig["budget_chart"]).update_layout(
    title="Net income by employment income under Blank Slate UBI",
))

In [15]:
format_fig(plotly_json_to_fig(fig["mtr_chart"]).update_layout(
    title="Marginal tax rates under Blank Slate UBI",
    yaxis_range=(-0.5, 1),
))